<a href="https://colab.research.google.com/github/carolineakello/ESMValTool/blob/main/flood_hazard_mapping_for_gkma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap --quiet

import ee, geemap
ee.Authenticate()
ee.Initialize(project='tough-nature-476114-u8')

Map = geemap.Map(center=[7.5, 125], zoom=8)

# Cloud mask function for Landsat 8
def cloud_mask(image):
    qa = image.select('QA_PIXEL')
    dilated = 1 << 1
    cirrus = 1 << 2
    cloud = 1 << 3
    shadow = 1 << 4
    mask = (qa.bitwiseAnd(dilated).eq(0)
              .And(qa.bitwiseAnd(cirrus).eq(0))
              .And(qa.bitwiseAnd(cloud).eq(0))
              .And(qa.bitwiseAnd(shadow).eq(0)))
    return (image.select(['SR_B.*'], ['B1','B2','B3','B4','B5','B6','B7'])
            .multiply(0.0000275)
            .add(-0.2)
            .updateMask(mask))

# datasets
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
srtm = ee.Image("USGS/SRTMGL1_003")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
roi = ee.FeatureCollection("projects/tough-nature-476114-u8/assets/KGMA_Boundary")

rainbow = ['blue', 'cyan', 'green', 'yellow', 'red']

#Water occurrence and permanent water
water = gsw.select('occurrence').clip(roi)
permanent = water.gt(80)

#Distance from permanent water
distance = (permanent.fastDistanceTransform(128, 'pixels')
            .sqrt()
            .multiply(30)
            .clip(roi))
only_distance = distance.updateMask(distance.neq(0).And(srtm.mask()))

distance_score = (only_distance
    .where(only_distance.gt(4000), 1)
    .where(only_distance.gt(3000).And(only_distance.lte(4000)), 2)
    .where(only_distance.gt(2000).And(only_distance.lte(3000)), 3)
    .where(only_distance.gt(1000).And(only_distance.lte(2000)), 4)
    .where(only_distance.lte(1000), 5))

# elevation & score
elevation = srtm.clip(roi)
elev_score = (elevation.updateMask(distance.neq(0))
    .where(elevation.gt(20), 1)
    .where(elevation.gt(15).And(elevation.lte(20)), 2)
    .where(elevation.gt(10).And(elevation.lte(15)), 3)
    .where(elevation.gt(5).And(elevation.lte(10)), 4)
    .where(elevation.lte(5), 5))

#Topographic Position Index & score
tpi = elevation.subtract(elevation.focalMean(5)).rename('TPI')
topo_score = (tpi.updateMask(distance.neq(0))
    .where(tpi.gt(0), 1)
    .where(tpi.gt(-2).And(tpi.lte(0)), 2)
    .where(tpi.gt(-4).And(tpi.lte(-2)), 3)
    .where(tpi.gt(-6).And(tpi.lte(-4)), 4)
    .where(tpi.lte(-8), 5))

#Landsat 8 processing
landsat8 = (l8.filterBounds(roi)
    .filterDate('2022-01-01', '2022-12-31')
    .map(cloud_mask)
    .median()
    .clip(roi))

band_map = {
    'red': landsat8.select('B4'),
    'nir': landsat8.select('B5'),
    'green': landsat8.select('B3')
}

# NDVI and vegetation hazard
ndvi = landsat8.expression('(nir - red) / (nir + red)', band_map).rename('NDVI')
veg_score = (ndvi.updateMask(distance.neq(0))
    .where(ndvi.gt(0.8), 1)
    .where(ndvi.gt(0.6).And(ndvi.lte(0.8)), 2)
    .where(ndvi.gt(0.4).And(ndvi.lte(0.6)), 3)
    .where(ndvi.gt(0.2).And(ndvi.lte(0.4)), 4)
    .where(ndvi.lte(0.2), 5))

# NDWI and wetness hazard
ndwi = landsat8.expression('(green - nir) / (green + nir)', band_map).rename('NDWI')
wet_score = (ndwi.updateMask(distance.neq(0))
    .where(ndwi.gt(0.6), 5)
    .where(ndwi.gt(0.2).And(ndwi.lte(0.6)), 4)
    .where(ndwi.gt(-0.2).And(ndwi.lte(0.2)), 3)
    .where(ndwi.gt(-0.6).And(ndwi.lte(-0.2)), 2)
    .where(ndwi.lte(-0.6), 1))

#Combine all hazard layers
flood_hazard = (distance_score
    .add(topo_score)
    .add(veg_score)
    .add(wet_score)
    .add(elev_score)
    .rename('Flood_hazard'))

flood_hazard_score = (flood_hazard
    .where(flood_hazard.gt(15), 5)
    .where(flood_hazard.gt(10).And(flood_hazard.lte(15)), 4)
    .where(flood_hazard.gt(5).And(flood_hazard.lte(10)), 3)
    .where(flood_hazard.gt(0).And(flood_hazard.lte(5)), 2)
    .where(flood_hazard.lte(0), 1))

#Add layers to map
Map.addLayer(water, {'min':0, 'max':100, 'palette':['white','cyan','blue']}, 'Water')
Map.addLayer(distance_score, {'min':1, 'max':5, 'palette':rainbow}, 'Distance Hazard')
Map.addLayer(elev_score, {'min':1, 'max':5, 'palette':rainbow}, 'Elevation Hazard')
Map.addLayer(topo_score, {'min':1, 'max':5, 'palette':rainbow}, 'Topographic Hazard')
Map.addLayer(veg_score, {'min':1, 'max':5, 'palette':rainbow}, 'Vegetation Hazard')
Map.addLayer(wet_score, {'min':1, 'max':5, 'palette':rainbow}, 'Wetness Hazard')
Map.addLayer(flood_hazard_score, {'min':1, 'max':5, 'palette':rainbow}, 'Flood Hazard Score')

# Add legend and controls
#Map.add_colorbar_branca(colors=rainbow, vmin=1, vmax=5, caption='Flood Hazard Level')
#Map.addLayerControl()
#Map
# Add legend and controls
Map.add_colorbar_branca(colors=rainbow, vmin=1, vmax=5, caption='Flood Hazard Level')
Map.addLayerControl()
Map



Map(center=[7.5, 125], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…